# 03 GraphRAG and AI Agent (Azure OpenAI Version)

In [ ]:
# !pip install -qU pymongo langchain_community langchain_openai langchain_mongodb voyageai

In [ ]:
import os
import voyageai
from pymongo import MongoClient

# 1. 환경 변수 및 키 설정
MONGODB_URI = ""
VOYAGE_API_KEY = ""

# Azure OpenAI 설정
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""

DB_NAME = "workshop_db"
COLLECTION_NAME = "books"
GRAPH_COLLECTION_NAME = "books_graph"
VECTOR_INDEX_NAME = f"{DB_NAME}_vector_search_index"

# 2. 클라이언트 초기화
# Voyage AI
vo = voyageai.Client(api_key=VOYAGE_API_KEY)

# MongoDB
mongodb_client = MongoClient(MONGODB_URI)
db = mongodb_client[DB_NAME]

## 1. 기존 데이터 로드 (From MongoDB)

이미 구축되어 있는 `books` 컬렉션에서 데이터를 가져옵니다. 이렇게 하면 Vector Search가 바라보는 데이터와 GraphRAG가 바라보는 데이터가 100% 일치함을 보장할 수 있습니다.

In [ ]:
from langchain_core.documents import Document

# 기존 books 컬렉션에서 데이터 조회 (줄거리가 있는 책만)
cursor = db[COLLECTION_NAME].find({"synopsis": {"$exists": True, "$ne": ""}})

documents = []
for doc in cursor:
    # GraphRAG 생성을 위한 문서 포맷팅
    content = f"Title: {doc['title']}\nSynopsis: {doc['synopsis']}"
    documents.append(Document(page_content=content, metadata={"title": doc['title']}))

print(f"Loaded {len(documents)} documents from existing '{COLLECTION_NAME}' collection.")

## 2. GraphRAG 구축 (New Collection)

기존 `books` 컬렉션은 건드리지 않고, `books_graph` 컬렉션에 노드와 엣지 정보를 저장합니다.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_mongodb.graphrag.graph import MongoDBGraphStore

# 1. LLM 초기화 (Azure OpenAI)
llm = AzureChatOpenAI(
    azure_deployment="gpt-5.1-chat",
    api_version="2024-12-01-preview",
)

# from_connection_string은 컬렉션 생성을 시도하므로, 이미 존재하면 에러가 발생합니다.
if GRAPH_COLLECTION_NAME in db.list_collection_names():
    print(f"Dropping existing collection: {GRAPH_COLLECTION_NAME}")
    db[GRAPH_COLLECTION_NAME].drop()

# 2. Graph Store 초기화 (새로운 컬렉션 사용)
graph_store = MongoDBGraphStore.from_connection_string(
    connection_string=MONGODB_URI,
    database_name=DB_NAME,
    collection_name=GRAPH_COLLECTION_NAME,
    entity_extraction_model=llm,
)

# 3. 그래프 데이터 생성
print("Building Knowledge Graph... (This involves LLM processing)")
graph_store.add_documents(documents)
print("Graph construction complete!")

In [ ]:
from langchain_core.prompts import PromptTemplate

FEW_SHOT_ENTITY_PROMPT = PromptTemplate.from_template(
    """
    You are an entity extractor. extract key entities (Person, Book, Concept, etc.) from the query.
    
    EXAMPLES:
    Input: "Who wrote Harry Potter?"
    Output: ["Harry Potter"]
    
    Input: "books about magic and wizards"
    Output: ["magic", "wizards"]
    
    Input: "What is the connection between A and B?"
    Output: ["A", "B"]

    RULES:
    1. Return ONLY a JSON list. 
    2. No markdown, no explanations.
    
    Input: "{input}"
    Output:
    """
)

graph_collection = db[GRAPH_COLLECTION_NAME]

# Graph Store 재연결 (새로운 프롬프트 적용)
existing_graph_store = MongoDBGraphStore(
    collection=graph_collection,
    entity_extraction_model=llm,
    entity_prompt=FEW_SHOT_ENTITY_PROMPT  # <--- 예시가 포함된 프롬프트 사용
)

def run_graph_rag(query: str):
    # 에러가 나도 죽지 않고 원인을 보여주도록 예외 처리 추가
    try:
        response = existing_graph_store.chat_response(query)
        return f"[GraphRAG Answer]\n{response.content}"
    except Exception as e:
        return f"[GraphRAG Error]\nLLM Output format error: {e}"

In [ ]:
query = "Who is Weston Hingler?"
# query = "What connects 'The Design of Everyday Things' to the concept of 'Product Design'?"
# query = "What do 'Ocean Sea' and 'Silk' have in common?"
# query = "Which book features a character who is an 'Illusionist' or 'Magician'?"
# query = "What books did J.R.R. Tolkien write?"

run_graph_rag(query)

In [ ]:
!pip install pyvis networkx

In [ ]:
import networkx as nx
from pyvis.network import Network

def visualize_graph(collection):
    docs = list(collection.find())
    
    def format_attributes(attrs):
        return "<br>".join(f"{k}: {', '.join(v)}" for k, v in attrs.items()) if attrs else ""
    
    G = nx.DiGraph()

    # Create nodes
    for doc in docs:
        node_id = str(doc["_id"])
        info = f"Type: {doc.get('type', '')}"
        if "attributes" in doc:
            attr_info = format_attributes(doc["attributes"])
            if attr_info:
                info += "<br>" + attr_info
        G.add_node(node_id, label=node_id, title=info.replace("<br>", "\n"))

    # Create edges
    for doc in docs:
        source = str(doc["_id"])
        rels = doc.get("relationships", {})
        targets = rels.get("target_ids", [])
        types = rels.get("types", [])
        attrs = rels.get("attributes", [])
        
        for i, target in enumerate(targets):
            edge_type = types[i] if i < len(types) else ""
            extra = attrs[i] if i < len(attrs) else {}
            edge_info = f"Relationship: {edge_type}"
            if extra:
                edge_info += "<br>" + format_attributes(extra)
            G.add_edge(source, str(target), label=edge_type, title=edge_info.replace("<br>", "\n"))

    # Build and configure network
    nt = Network(notebook=True, cdn_resources='in_line', width="800px", height="600px", directed=True)
    nt.from_nx(G)
    nt.set_options('''
    var options = {
      "interaction": {
        "hover": true,
        "tooltipDelay": 200
      },
      "nodes": {
        "font": {"multi": "html"}
      },
      "physics": {
        "repulsion": {
          "nodeDistance": 300,
          "centralGravity": 0.2,
          "springLength": 200,
          "springStrength": 0.05,
          "damping": 0.09
        }
      }
    }
    ''')

    return nt.generate_html()

In [ ]:
from IPython.display import HTML, display

collection = db[GRAPH_COLLECTION_NAME]
html = visualize_graph(collection)

display(HTML(html))

## 4. 비교를 위한 준비: 텍스트 임베딩 추가 (Text Embedding)

우리는 Part 1에서 책 표지(이미지)를 벡터화하여 저장했습니다.
하지만 지금 우리가 하려는 비교 실험은 "줄거리를 기반으로 내용을 묻는 것"입니다.
이미지 벡터로는 줄거리 질문에 정확히 답할 수 없으므로, 비교가 되지 않습니다.

따라서, 기존 이미지 벡터(`embedding`)는 그대로 두고, 줄거리 텍스트를 벡터화하여 `text_embedding`이라는 새로운 필드에 저장하겠습니다.

In [ ]:
from tqdm import tqdm

print("Generating Text Embeddings for Fair Comparison...")

# 1. 줄거리가 있는 책들만 가져옵니다.
cursor = db[COLLECTION_NAME].find({"synopsis": {"$exists": True, "$ne": ""}})
docs_to_update = list(cursor)

# 2. 텍스트 임베딩 생성 및 'text_embedding' 필드에 저장
for doc in tqdm(docs_to_update, desc="Updating text_embedding"):
    # GraphRAG와 동일한 소스 텍스트 사용
    text_content = f"Title: {doc['title']}\nSynopsis: {doc['synopsis']}"
    
    # 텍스트 임베딩 생성 (Voyage AI)
    new_embedding = vo.multimodal_embed(
        inputs=[[text_content]], 
        model="voyage-multimodal-3", 
        input_type="document"
    ).embeddings[0]
    
    # DB 업데이트: 기존 'embedding'은 건드리지 않고 'text_embedding' 필드를 새로 만듭니다.
    db[COLLECTION_NAME].update_one(
        {"_id": doc["_id"]},
        {"$set": {"text_embedding": new_embedding}}
    )

print("Text embeddings saved to 'text_embedding' field!")

새로운 필드(text_embedding)가 생겼으니, 이 필드를 전용으로 검색할 수 있는 새로운 인덱스(vector_index_text)가 필요합니다.

In [ ]:
from utils import create_index, check_index_ready

TEXT_VECTOR_INDEX_NAME = f"{DB_NAME}_vector_index_text"

# 인덱스 정의 (path가 'text_embedding'인 것이 핵심입니다)
text_vector_index_model = {
    "name": TEXT_VECTOR_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "text_embedding",
                "numDimensions": 1024,
                "similarity": "cosine"
            }
        ]
    }
}


# `create_index` 함수를 사용하여 `collection` 컬렉션에 위 정의대로 벡터 검색 인덱스를 생성합니다.
create_index(collection, TEXT_VECTOR_INDEX_NAME, text_vector_index_model)

# 진행하기 전에 `check_index_ready` 함수를 사용하여 인덱스가 생성되었고 'READY' 상태인지 확인합니다.
check_index_ready(collection, TEXT_VECTOR_INDEX_NAME)

## 3. 비교 실험: Vector Search vs GraphRAG

이제 두 가지 검색 방식을 비교해 봅니다.
* **Vector Search:** Part 1에서 만든 `books` 컬렉션 + `vector_index` 사용
* **GraphRAG:** 방금 만든 `books_graph` 컬렉션 사용

In [ ]:
# [Standard RAG] Vector Search 결과를 LLM에게 전달하여 답변 생성
def run_standard_rag(query: str):
    # 1. 임베딩 생성 (Voyage AI)
    query_embedding = vo.multimodal_embed(
        inputs=[[query]], model="voyage-multimodal-3", input_type="query"
    ).embeddings[0]

    # 2. Vector Search 실행 (Top 3)
    pipeline = [
        {
            "$vectorSearch": {
                "index": TEXT_VECTOR_INDEX_NAME,
                "path": "text_embedding",
                "queryVector": query_embedding,
                "numCandidates": 50,
                "limit": 3
            }
        },
        {"$project": {"_id": 0, "title": 1, "synopsis": 1}}
    ]
    results = list(db[COLLECTION_NAME].aggregate(pipeline))

    # 3. 검색된 문서가 없을 경우
    if not results:
        return "[Standard RAG Answer]\nI couldn't find any relevant documents."

    # 4. Context 구성 (검색된 줄거리 합치기)
    context_text = ""
    for doc in results:
        context_text += f"Title: {doc['title']}\nSynopsis: {doc.get('synopsis', 'No synopsis available')}\n\n"

    # 5. LLM에게 답변 요청 (Azure OpenAI)
    # GraphRAG와 공정한 비교를 위해 동일한 LLM(llm 객체)을 사용합니다.
    prompt = f"""
    You are a helpful assistant. Answer the question based ONLY on the following context.
    If the answer is not in the context, say "I don't know".

    Context:
    {context_text}

    Question: {query}
    """
    
    response = llm.invoke(prompt)
    return f"[Standard RAG Answer]\n{response.content}"

### 비교 시나리오: 복합적인 관계 추론
벡터 검색은 키워드 매칭에는 강하지만, "A와 B의 관계"나 "특정 주제를 다루는 책의 갈등" 같은 질문에는 약할 수 있습니다.

In [ ]:
# --- [최종 비교 실행] ---
# query = "Who is Weston Hingler?"
# query = "What connects 'The Design of Everyday Things' to the concept of 'Product Design'?"
# query = "What do 'Ocean Sea' and 'Silk' have in common?"
# query = "Which book features a character who is an 'Illusionist' or 'Magician'?"
# query = "What books did J.R.R. Tolkien write?"
query = "Who wrote the books The Hobbit?"
# query = "What businesses are owned by Weston Hingler's parents?"
# query = "Give me a summary of 'Prodigal Summer'."

print("="*50)
print(f"QUERY: {query}")
print("="*50)

# 1. Standard RAG (Text Vector)
print(run_standard_rag(query))
print("-" * 50)

# 2. GraphRAG (Graph)
print(run_graph_rag(query)) # 앞서 정의한 run_graph_rag 재사용
print("="*50)

In [ ]:
query = "Give me a summary of 'Prodigal Summer'."

print("="*50)
print(f"QUERY: {query}")
print("="*50)

# 1. Standard RAG (Text Vector)
# print(run_standard_rag(query))
print("-" * 50)

# 2. GraphRAG (Graph)
print(run_graph_rag(query))

## 4. AI Agent (GraphRAG Tool 활용)

GraphRAG를 에이전트의 도구로 장착하여, 필요할 때 지식 그래프를 조회하도록 만듭니다.

In [ ]:
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages

# 1. 도구(Tools) 정의

# Azure OpenAI LLM 정의
llm = AzureChatOpenAI(
    azure_deployment="gpt-5.1-chat",
    api_version="2024-12-01-preview",
)

@tool
def vector_search_tool(query: str) -> str:
    """
    Useful for answering questions about specific plot details, summaries, text excerpts, 
    or general descriptions of books. Use this for 'What is the book about?' type questions.
    """
    # 앞서 정의한 Standard RAG 함수 재사용
    return run_standard_rag(query)

@tool
def graph_search_tool(query: str) -> str:
    """
    Useful for answering questions about relationships between characters, authors, 
    business ownership, family trees, or specific connections. 
    Use this for 'Who is related to whom?' or 'Who owns what?' type questions.
    """
    # 앞서 정의한 Graph RAG 함수 재사용
    return run_graph_rag(query)

# 도구 리스트
tools = [vector_search_tool, graph_search_tool]

# 2. LLM에 도구 바인딩 (Bind Tools)
llm_with_tools = llm.bind_tools(tools)

# 3. 그래프 상태(State) 정의
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]

# 4. 노드(Nodes) 정의
def agent_node(state: AgentState):
    """현재 상태를 보고 LLM이 답변할지 도구를 호출할지 결정"""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# 도구 실행 노드 (LangGraph Prebuilt 사용)
tool_node = ToolNode(tools)

# 5. 조건부 엣지(Conditional Edge) 정의
def should_continue(state: AgentState):
    """LLM의 응답에 tool_calls가 있으면 도구 실행 노드로, 없으면 종료"""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

# 6. 그래프 구축 (Build Graph)
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# 엣지 연결
workflow.set_entry_point("agent") # 시작 -> 에이전트
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)
workflow.add_edge("tools", "agent") # 도구 실행 후 -> 다시 에이전트(결과 종합)

# 컴파일
app = workflow.compile()

In [ ]:
app

In [ ]:
from langchain_core.messages import HumanMessage

def run_agent_test(query: str):
    """
    에이전트에게 질문을 던지고, 어떤 도구를 선택했는지와 최종 답변을 출력합니다.
    """
    print("=" * 60)
    print(f"🗣️ User Question: {query}")
    print("-" * 60)

    final_response = None

    # LangGraph 스트리밍 실행
    # messages 키에 HumanMessage를 담아서 그래프에 전달합니다.
    for event in app.stream({"messages": [HumanMessage(content=query)]}):
        
        for node_name, state in event.items():
            # 1. 에이전트 노드: 도구 사용 여부 판단
            if node_name == "agent":
                last_msg = state["messages"][-1]
                if last_msg.tool_calls:
                    tool_name = last_msg.tool_calls[0]['name']
                    print(f"🤖 Agent Thought: 'I need to use [{tool_name}] to answer this.'")
                    print(f"👉 Calling Tool: {tool_name}...")
            
            # 2. 도구 노드: 도구 실행 완료
            elif node_name == "tools":
                print("🔧 Tool Output: Data received successfully.")

            # 마지막 메시지 추적 (최종 답변 출력을 위해)
            if "messages" in state:
                final_response = state["messages"][-1].content

    # 최종 답변 출력
    print("-" * 60)
    print(f"💡 Final Answer:\n{final_response}")
    print("=" * 60 + "\n")

In [ ]:
# [테스트 1] 줄거리/요약 질문 -> Vector Search 기대
# "Prodigal Summer"의 줄거리는 텍스트에만 있으므로 vector_search_tool을 선택해야 합니다.
run_agent_test("Give me a summary of 'Prodigal Summer'.")

# [테스트 2] 관계/구조 질문 -> Graph Search 기대
# "Weston Hingler"의 부모님이 소유한 사업체는 그래프 관계에 있으므로 graph_search_tool을 선택해야 합니다.
run_agent_test("What businesses are owned by Weston Hingler's parents?")

### n8n 실습을 위한 Vector Search Index 생성

In [ ]:
N8N_VECTOR_INDEX_NAME = f"{DB_NAME}_n8n"

model = {
    "name": N8N_VECTOR_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 1536,
                "similarity": "cosine"
            }
        ]
    }
}

collection = db["inven"]

create_index(collection, N8N_VECTOR_INDEX_NAME, model)